In [1]:
import json
with open('./data/CVPR_2022_NAS_Track2_train.json', 'r') as f:
    train_data = json.load(f)

In [2]:
import pandas as pd
import numpy as np

In [3]:
target_cols = ['cplfw_rank', 'market1501_rank', 'dukemtmc_rank', 'msmt17_rank', 'veri_rank', 'vehicleid_rank', 'veriwild_rank', 'sop_rank']

In [4]:
def get_df(train_data):
    ret = []
    for k, v in train_data.items():
        tmp = list(v['arch'])
        tmp1 = []
        for c in target_cols:
            tmp1.append(v[c])
        ret.append(tmp+tmp1+[k,v['arch']])
    retf = pd.DataFrame(ret,columns=[f'col{_}' for _ in range(len(tmp))]+target_cols+['id','arch'])
    retf['col0'] = retf['col0'].map({'l':1, 'j':2, 'k':3})
    int_cols = [x for x in retf.columns if x not in ['id','arch']]
    retf[int_cols] = retf[int_cols].astype(float)
    return retf

In [5]:
train_df = get_df(train_data)

In [6]:
def get_step(train_df):
    res0 = []
    res1 = []
    res2 = []
    res3 = []
    res4 = []
    map_={'l':1, 'j':2, 'k':3}
    time_step=12
    head_ = set([1+_*3 for _ in range(12)])
    mlp_ = set([2+_*3 for _ in range(12)])
    emb_ = set([3+_*3 for _ in range(12)])
    depth_ = set([0])
    for item in train_df.arch:
        ret = np.array(list(item[1:]),dtype=np.float32).reshape(-1,3)
        res0.append([1 if x in head_ else 0 for x in range(37)])
        res1.append([1 if x in mlp_ else 0 for x in range(37)])
        res2.append([1 if x in emb_ else 0 for x in range(37)])
        res3.append([1 if x in depth_ else 0 for x in range(37)])
        res4.append([map_[item[0]]]+list(np.array(list(item[1:]),dtype=np.float32)))
        
    train_df['head'] = res0
    train_df['mlp'] = res1
    train_df['emb'] = res2
    train_df['depth'] = res3
    train_df['all_emb'] = res4

In [7]:
get_step(train_df)

In [8]:
from sklearn.model_selection import KFold
from tqdm import tqdm

In [20]:
#定义数据
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self,df,use_cols,target_cols,show=0):
        self.df = df
        self.show = show
        self.use_cols = use_cols
        self.target_cols = target_cols

        self.prepare_data()
        
    def __len__(self):
        return self.df.shape[0]
    
    def prepare_data(self):
        self.y = self.df[self.target_cols].values
        self.y0 = self.df[[self.target_cols[0]]].values
        self.y1 = self.df[[self.target_cols[1]]].values
        self.y2 = self.df[[self.target_cols[2]]].values
        self.y3 = self.df[[self.target_cols[3]]].values
        self.y4 = self.df[[self.target_cols[4]]].values
        self.y5 = self.df[[self.target_cols[5]]].values
        self.y6 = self.df[[self.target_cols[6]]].values
        self.y7 = self.df[[self.target_cols[7]]].values
        
        uc = ['all_emb']
        tmp_dt = {}
        for c in uc:
            tmp_dt[c] = np.array(self.df[c].tolist())
        self.inputs = np.concatenate([tmp_dt[c][:, None] for c in uc], 1).transpose(0, 2, 1)

        
        if self.show==1:
            print('inputs_shape',self.inputs.shape)
            print('y_shape',self.y.shape)
        
    def __getitem__(self, idx):
        data = {
            "input": torch.tensor(self.inputs[idx], dtype=torch.float),
            "y": torch.tensor(self.y[idx], dtype=torch.float),
            "y0": torch.tensor(self.y0[idx], dtype=torch.float),
            "y1": torch.tensor(self.y1[idx], dtype=torch.float),
            "y2": torch.tensor(self.y2[idx], dtype=torch.float),
            "y3": torch.tensor(self.y3[idx], dtype=torch.float),
            "y4": torch.tensor(self.y4[idx], dtype=torch.float),
            "y5": torch.tensor(self.y5[idx], dtype=torch.float),
            "y6": torch.tensor(self.y6[idx], dtype=torch.float),
            "y7": torch.tensor(self.y7[idx], dtype=torch.float),
        }
        
        return data

In [10]:
use_cols = [x for x in train_df.columns if 'col' in x]

In [11]:
len(use_cols)

37

In [12]:
import gc
import os
import time
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup


def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    
def count_parameters(model, all=False):
    if all:
        return sum(p.numel() for p in model.parameters())
    else:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)


def save_model_weights(model, modelpath, filename):
    torch.save(model.state_dict(), modelpath+filename)
    return f"\n -> Save weights to {modelpath+filename}\n"

In [13]:
import scipy

In [14]:
def compute_metric(pred, y):
    corr = []
    if pred.shape[1]>2:
        for i in range(8):
            corr.append(scipy.stats.stats.kendalltau(pred[:, i], y[:, i])[0])
    else:
        corr.append(scipy.stats.stats.kendalltau(pred, y)[0])
    return np.array(corr)

    
class CVPRLoss_tanh(nn.Module):
    # kendall tanh
    def __call__(self, pred, y):
        pred = pred
        y = y
        return 1-torch.cat(
            [self.get_score(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,-1) for i in range(y.shape[1])]
        ).reshape(1, -1)
    
    def get_score(self, actuals, preds):
        sa = actuals.argsort()[0]
        tmp = preds.index_select(1, sa.int())[0]
        score = torch.cat([((tmp[i:]-tmp[i-1])*1).tanh() for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
        return score1
    

In [15]:
def train(model, 
        train_dataset, 
        val_dataset, 
        verbose=20, 
        fold_=0,
        modelname='MLP_base',
        modelpath=r'./model'+'//',
        input='input',
        y='y',
        early_stop_round=60,
        debug=False):
    
    print(f'Model parameters count: {count_parameters(model)}')
    #数据加载
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        drop_last=True,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE_TEST,
        shuffle=False,
        drop_last=False,
        pin_memory=True
    )
    print(f'train batch num: {len(train_loader)}')
    print(f'val batch num: {len(val_loader)}')
            
    # Optimizer
    optimizer = getattr(torch.optim, optim)(model.parameters(), lr=LR)
    # Scheduler
    num_warmup_steps = int(0.1 * EPOCHS * len(train_loader))
    num_training_steps = int(EPOCHS * len(train_loader))
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps, num_training_steps
    )
    print(f'optim: {optim}, lr: {LR}, warmup_steps: {num_warmup_steps}')
    
    print(f'early stopping round: {early_stop_round}\n')
    #train
    score_best=0
    bst_epoch=0
    first_epoch_eval=0
    for epoch in range(EPOCHS):
        if epoch > early_stop_round and (epoch - bst_epoch > early_stop_round):
            print(f'early stopping.')
            break
        
        model.train()
        model.zero_grad()
        start_time = time.time()

        avg_loss = 0
        for data in train_loader:
            pred = model(data[input].to(device))
#             print(pred.shape,data['y'].shape)

            loss = loss_fct(
                pred,
#                 data[y+'_quantile'].to(device),
                data[y].to(device),
            ).mean()

            loss.backward()
            avg_loss += loss.item() / len(train_loader)

            optimizer.step()
            scheduler.step()

            optimizer.zero_grad(set_to_none=True)

        #VAL
        model.eval()
        mae, avg_val_loss = 0, 0
        preds = []

        with torch.no_grad():
            for data in val_loader:
                pred = model(data[input].to(device))

                loss = loss_fct(
                    pred,
                    data[y].to(device)
                ).mean()

                avg_val_loss += loss.item() / len(val_loader)

                preds.append(pred.detach().cpu().numpy())

        preds = np.concatenate(preds, 0)
        if y=='y':
            mae = compute_metric(preds,val_dataset.df[target_cols].values).mean()
        else:
            mae = compute_metric(preds,val_dataset.df[[target_cols[int(y.replace('y',''))]]].values).mean()
        
        elapsed_time = time.time() - start_time
        if (epoch + 1) % verbose == 0:
            elapsed_time = elapsed_time * verbose
            lr = scheduler.get_last_lr()[0]
    #         lr=LR
            print(
                f"Epoch {epoch + 1:02d}/{ EPOCHS:02d} \t lr={lr:.1e}\t t={elapsed_time:.0f}s \t"
                f"loss={avg_loss:.4f}",
                end="\t",
            )

            if (epoch + 1 >= first_epoch_eval) or (epoch + 1 == EPOCHS):
                print(f"val_loss={avg_val_loss:.4f}\tcorr={mae:.4f}")
            else:
                print("")
                
        #保存最优模型
        if mae>score_best:
            bst = save_model_weights(model, modelpath, f'{modelname}_{fold_}.pt')
            score_best = mae
            bst_epoch = epoch
            if y=='y':
                bst_list = compute_metric(preds,val_dataset.df[target_cols].values)
            else:
                bst_list = compute_metric(preds,val_dataset.df[[target_cols[int(y.replace('y',''))]]].values).mean()
            bst_preds = preds
    print(f'best score {score_best}, best epoch: {bst_epoch}, {bst} ' )
    print(bst_list,'\n\n')
    del (val_loader, train_loader, loss, data, pred)
    gc.collect()
    torch.cuda.empty_cache()
    
    return bst_preds, bst_list


In [16]:
class CVPRModel(nn.Module):
    def __init__(
        self,
        input_dim=3,
        num_classes=8,
        time_step=12,
        bi=True
    ):
        super(CVPRModel,self).__init__()
        
        self.bi_num = 2 if bi else 1
        
        self.MLP = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU()
        )

        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=256, 
            nhead=2, 
            dropout=0,
            dim_feedforward=256*4
        )
        self.encoder = nn.TransformerEncoder(
            encoder_layer=self.encoder_layer, 
            num_layers=1
        )
        
        
        self.decoder_layer = nn.TransformerDecoderLayer(
            d_model=256, 
            nhead=2,
            dropout=0,
            dim_feedforward=256*4
        )      
        self.decoder = nn.TransformerDecoder(
            decoder_layer=self.decoder_layer, 
            num_layers=1
        )
        
        
        self.Logits = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.Linear(256*time_step, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x):
        x1 = self.MLP(x)
        enc = self.encoder(x1)
        dec = self.decoder(x1, enc)
        
        pred = self.Logits(dec)
        return pred

In [18]:
#参数设置
seed = 666
BATCH_SIZE = 384
BATCH_SIZE_TEST = 128
EPOCHS = 300
LR = 0.0001
optim = "Adam"
device = "cuda" if torch.cuda.is_available() else "cpu"
modelpath = r'./model'+'//'

In [21]:
# K折训练，同时
loss_fct = CVPRLoss_tanh()
k=5
scoref = []
skf = KFold(n_splits=k, shuffle=False)
for index, (train_index, test_index) in enumerate(skf.split(train_df)):   
    print(f'FOLD {index}')
    train0 = train_df.iloc[train_index]
    val0 = train_df.iloc[test_index]   
    train_dataset = MyDataset(train0, use_cols, target_cols)
    val_dataset = MyDataset(val0, use_cols, target_cols)
    print(f'train size: {len(train0)}, val size: {len(val0)}')

    modelname = f'transformer_encoder_decoder_tanh'
    seed_everything(seed)
    model = CVPRModel(input_dim=1,
                    num_classes=8,
                    bi=True,
                    time_step=37
                   ).to(device)
    preds,_ = train(model, 
                train_dataset, 
                val_dataset, 
                verbose=100, 
                fold_=index,
                modelname=modelname,
                modelpath=modelpath,
                input='input',
                y='y',
                debug=False
                 )
    scoref.append(_)
scoreff = scoref
np.round(np.array(scoreff).mean(1).mean(), 5), [round(x, 5) for x in np.array(scoreff).mean(0)]

FOLD 0
train size: 400, val size: 100
Model parameters count: 25548808
train batch num: 1
val batch num: 1
optim: Adam, lr: 0.0001, warmup_steps: 30
early stopping round: 60

Epoch 100/300 	 lr=7.4e-05	 t=41s 	loss=0.0923	val_loss=0.2270	corr=0.7798
early stopping.
best score 0.785858585858586, best epoch: 66, 
 -> Save weights to ./model//transformer_encoder_decoder_tanh_0.pt
 
[0.28       0.89737374 0.89292929 0.94909091 0.88040404 0.6820202
 0.91959596 0.78545455] 


FOLD 1
train size: 400, val size: 100
Model parameters count: 25548808
train batch num: 1
val batch num: 1
optim: Adam, lr: 0.0001, warmup_steps: 30
early stopping round: 60

Epoch 100/300 	 lr=7.4e-05	 t=41s 	loss=0.0923	val_loss=0.2323	corr=0.7742
early stopping.
best score 0.7763131313131314, best epoch: 82, 
 -> Save weights to ./model//transformer_encoder_decoder_tanh_1.pt
 
[0.27717172 0.86868687 0.89979798 0.95393939 0.88646465 0.64080808
 0.88808081 0.79555556] 


FOLD 2
train size: 400, val size: 100
Model para

(0.77898,
 [0.25519, 0.86675, 0.89358, 0.952, 0.89236, 0.66723, 0.91265, 0.79208])

In [22]:
np.round(np.array(scoreff).mean(1).mean(), 5), [round(x, 5) for x in np.array(scoreff).mean(0)]

(0.77898,
 [0.25519, 0.86675, 0.89358, 0.952, 0.89236, 0.66723, 0.91265, 0.79208])

In [23]:
#加载test
with open('./data/CVPR_2022_NAS_Track2_test.json', 'r') as f:
    test_data = json.load(f)

In [24]:
test_df = get_df(test_data)
get_step(test_df)

In [30]:
def pred_cuda(test_df, model):
    #获得预测
    test_dataset = MyDataset(test_df, use_cols, target_cols, 0)
    test_loader = DataLoader(
        test_dataset,
        batch_size=1024*2,
        shuffle=False,
        drop_last=False,
        pin_memory=True
    )

    preds = []
    model.eval()
    with torch.no_grad():
        for data in tqdm(test_loader):
            pred = model(data['input'].to(device))
            preds.append(pred.detach().cpu().numpy())
    preds = np.concatenate(preds, 0)
    print(preds.shape)
    
    del test_dataset, test_loader, model;
    gc.collect()
    torch.cuda.empty_cache()
    return preds
        
def predict_all(test_df, k=5, modelname = f'transformer_encoder_decoder_hardtanh'):
        
    print(f'Model {modelname}')
    
    k_list = [0,1,2,3,4]

    cols = []
    for fold_ in range(k):
        if fold_ not in k_list:
            continue
        model = CVPRModel(input_dim=1,
                    num_classes=8,
                    bi=True,
                    time_step=37
                   ).to(device)
        model.load_state_dict(torch.load(modelpath+f'{modelname}_{fold_}.pt'))

        pred_ = pred_cuda(test_df, model)
        tmp_c = [f'{target}_{fold_}' for target in target_cols]
        test_df[tmp_c] = pred_
        cols += tmp_c
    #get rank
    print(cols)
    test_df[cols] = test_df[cols].rank()
    for c in target_cols:
        test_df[c] = test_df[[f'{c}_{fold_}' for fold_ in k_list]].mean(axis=1)
        test_df[c] = test_df[c].rank()

In [31]:
k, modelname

(5, 'transformer_encoder_decoder_tanh')

In [32]:
model_dt = predict_all(test_df, k=5, modelname=f'transformer_encoder_decoder_tanh')

Model transformer_encoder_decoder_tanh


C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


  0%|          | 0/49 [00:00<?, ?it/s]

(99500, 8)


  0%|          | 0/49 [00:00<?, ?it/s]

...

(99500, 8)


  0%|          | 0/49 [00:00<?, ?it/s]

(99500, 8)


  0%|          | 0/49 [00:00<?, ?it/s]

(99500, 8)
['cplfw_rank_0', 'market1501_rank_0', 'dukemtmc_rank_0', 'msmt17_rank_0', 'veri_rank_0', 'vehicleid_rank_0', 'veriwild_rank_0', 'sop_rank_0', 'cplfw_rank_1', 'market1501_rank_1', 'dukemtmc_rank_1', 'msmt17_rank_1', 'veri_rank_1', 'vehicleid_rank_1', 'veriwild_rank_1', 'sop_rank_1', 'cplfw_rank_2', 'market1501_rank_2', 'dukemtmc_rank_2', 'msmt17_rank_2', 'veri_rank_2', 'vehicleid_rank_2', 'veriwild_rank_2', 'sop_rank_2', 'cplfw_rank_3', 'market1501_rank_3', 'dukemtmc_rank_3', 'msmt17_rank_3', 'veri_rank_3', 'vehicleid_rank_3', 'veriwild_rank_3', 'sop_rank_3', 'cplfw_rank_4', 'market1501_rank_4', 'dukemtmc_rank_4', 'msmt17_rank_4', 'veri_rank_4', 'vehicleid_rank_4', 'veriwild_rank_4', 'sop_rank_4']


In [33]:
test_df[target_cols] = test_df[target_cols].astype(int)-1

In [34]:
test_df[target_cols].describe()

,cplfw_rank,market1501_rank,dukemtmc_rank,msmt17_rank,veri_rank,vehicleid_rank,veriwild_rank,sop_rank
count,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000
mean,49749.419427,49749.418402,49749.419116,49749.418804,49749.418382,49749.419226,49749.417889,49749.419749
std,28723.319720,28723.320380,28723.318907,28723.321470,28723.320690,28723.320862,28723.320015,28723.320291
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,24874.000000,24874.750000,24874.750000,24874.750000,24874.750000,24874.750000,24874.750000,24874.750000
50%,49749.500000,49749.500000,49749.500000,49749.500000,49749.500000,49749.500000,49749.500000,49749.500000
75%,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000
max,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000


In [35]:
modelname

'transformer_encoder_decoder_tanh'

In [36]:
# to_sub
def to_sub(test_df, test_data, name='CVPR_2022_lgb_score'):
    for i in tqdm(test_df[['id']+target_cols].values):
        id_ = i[0]
        for k,v in enumerate(target_cols):
            k += 1
            test_data[id_][v] = i[k]
            
    with open(f'./sub/{name}.json', 'w') as f:
        json.dump(test_data, f)

In [37]:
to_sub(test_df, test_data, name='CVPR_2022_transformer_encoder_decoder_tanh')

C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/99500 [00:00<?, ?it/s]